In [1]:
import yaml

data = {
    'path': "car_dataset/archive",
    'test': 'images/test/',
    'train': 'images/train/',
    'val': 'images/test/',
    'nc': 1,
    'names': ['car']
}

with open('car_dataset/data.yaml', 'w') as file:
    yaml.dump(data, file, sort_keys=False)

In [2]:
import os
import random
import shutil

# Define paths
images_dir = "car_dataset/images"
labels_dir = "car_dataset/labels"

image_val = os.path.join(images_dir, "val")
image_test = os.path.join(images_dir, "test")

label_val = os.path.join(labels_dir, "val")
label_test = os.path.join(labels_dir, "test")

# Create test directories if they don't exist
os.makedirs(image_test, exist_ok=True)
os.makedirs(label_test, exist_ok=True)

# Get all image files from val (assuming .png/.jpg)
val_images = [os.path.join(image_val, f) for f in os.listdir(image_val) if f.endswith((".png", ".jpg", ".jpeg"))]
random.shuffle(val_images)  # Shuffle randomly

# Calculate 50% split
split_idx = len(val_images) // 2
test_images = val_images[:split_idx]
remaining_val_images = val_images[split_idx:]

# Move images + labels to test/val
for img_path in test_images:
    # Get corresponding label (e.g., 1.jpg -> 1.txt)
    filename = os.path.basename(img_path)
    label_name = os.path.splitext(filename)[0] + ".txt"
    label_path = os.path.join(label_val, label_name)

    # Move image to test
    shutil.move(img_path, os.path.join(image_test, filename))
    
    # Move label to test (if exists)
    if os.path.exists(label_path):
        shutil.move(label_path, os.path.join(label_test, label_name))
    else:
        print(f"Warning: Label {label_path} not found!")

print(f"Moved {len(test_images)} images (and labels) from val → test.")
print(f"Remaining val files: {len(remaining_val_images)} images.")

Moved 49 images (and labels) from val → test.
Remaining val files: 50 images.


In [3]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")
total_layers = sum(1 for _ in model.model.named_parameters())  # Count layers
print(f"Total trainable layers: {total_layers}")

Total trainable layers: 184


In [1]:
from ultralytics import YOLO

# Load the YOLOv8 model (pre-trained on COCO)
model = YOLO('yolov8s.pt')  
path_to_data_yaml = "car_dataset/data.yaml"
# Train the model on the custom dataset
results = model.train(
    data=path_to_data_yaml,  
    epochs=50,                      # Number of epochs to train
    batch=100,                       # Batch size
    imgsz=640,                      # Image size
    #freeze=70,                      # Freeze backbone
    plots=True,                     # Generate plots
    # resume=True,
    device=0                        # Use GPU (set -1 for CPU)
)

New https://pypi.org/project/ultralytics/8.3.102 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.96  Python-3.11.5 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=car_dataset/data.yaml, epochs=50, time=None, patience=100, batch=100, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_f

train: Scanning D:\VsCode\Computer_Vision\CNN.YOLO\car_dataset\labels\train.cache... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<?, ?it/s]
val: Scanning D:\VsCode\Computer_Vision\CNN.YOLO\car_dataset\labels\test.cache... 49 images, 0 backgrounds, 0 corrupt: 100%|██████████| 49/49 [00:00<?, ?it/s]


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00078125), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      20.5G      1.331      2.333     0.9832       1997        640: 100%|██████████| 4/4 [01:26<00:00, 21.61s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.58s/it]

                   all         49        317      0.149      0.628      0.116     0.0768


...

      50/50      20.7G      0.326     0.2001     0.7762       1095        640: 100%|██████████| 4/4 [01:40<00:00, 25.02s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.88s/it]

                   all         49        317      0.819      0.913      0.914      0.758



50 epochs completed in 1.470 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train\weights\best.pt, 22.5MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.96  Python-3.11.5 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.13it/s]


                   all         49        317      0.828      0.898      0.924      0.767
Speed: 0.2ms preprocess, 14.4ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\train


In [ ]:
# Load your trained model
model = YOLO('runs/detect/train/weights/best.pt')  # Path to your best.pt
path_to_data_yaml = "C:/Users/Sid/Documents/computer_vision/CNN.YOLO/datasets/vehicle_dataset/data.yaml"

# Evaluate on the test set
results = model.val(
    data=path_to_data_yaml,
    split='test',  # Force evaluation on the test set (not validation)
    batch=16,
    imgsz=256,
    plots=True,
    verbose=True,
    device=0,       # GPU
    save_json=True  # Optional: Save results to JSON for analysis
)